<a href="https://colab.research.google.com/github/DavidP0011/apps_functions/blob/main/app_transc_02_whisper_to_spreadsheet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INICIALIZACIÓN

In [ ]:
#@title install_dpm_repos()
def install_dpm_repos(config: dict) -> None:
    """
    Instala repositorios DPM y lista las funciones de cada paquete instalado.

    El diccionario de configuración debe tener la clave:
        - github_repo_url_list: Lista de URLs de repositorios GitHub
          Ejemplo:
            {
                "github_repo_url_list": [
                  "https://github.com/DavidP0011/common_functions",
                  "https://github.com/DavidP0011/utils_functions",
                  "https://github.com/DavidP0011/etl_functions"
                  ]
            }

    La función procede en dos pasos para cada repositorio:
      1. Se instala de manera “normal” (pip install --upgrade --no-cache-dir git+<repo_url>)
         para instalar las dependencias si faltan.
      2. Se fuerza la reinstalación de los archivos .py con --force-reinstall y --no-deps
         (pip install --upgrade --force-reinstall --no-deps --no-cache-dir git+<repo_url>).

    Luego, se detecta dinámicamente el nombre del paquete instalado y se recorre
    cada paquete para imprimir las funciones definidas.
    """
    import subprocess
    import sys
    import importlib
    import pkgutil
    import inspect

    # Para detectar distribuciones y top-level packages
    try:
        import importlib.metadata as metadata
    except ImportError:
        # Si la versión de Python es muy antigua:
        import importlib_metadata as metadata

    def _validate_config(cfg: dict) -> list:
        """Valida que la configuración tenga la clave 'github_repo_url_list' con una lista."""
        if "github_repo_url_list" not in cfg:
            raise ValueError("[VALIDATION [ERROR ❌]] La configuración debe contener la clave 'github_repo_url_list'.")
        if not isinstance(cfg["github_repo_url_list"], list):
            raise ValueError("[VALIDATION [ERROR ❌]] La clave 'github_repo_url_list' debe ser una lista de URLs.")
        return cfg["github_repo_url_list"]

    def _install_repo(repo_url: str) -> None:
        """
        Instala un repositorio en dos pasos:
          1. Instala el repositorio de forma normal (para que se instalen las dependencias si aún no están).
          2. Fuerza la reinstalación del código (los archivos .py) sin tocar las dependencias (--no-deps).
        """
        print(f"\n[START ▶️] Instalación inicial (con dependencias) del repositorio: {repo_url}", flush=True)
        cmd_normal = [
            sys.executable, "-m", "pip", "install", "-q",
            "--upgrade", "--no-cache-dir",
            f"git+{repo_url}"
        ]
        try:
            subprocess.check_call(cmd_normal)
            print(f"[SUCCESS ✅] Instalación inicial completada.", flush=True)
        except subprocess.CalledProcessError as e:
            print(f"[ERROR ❌] Falló la instalación inicial del repositorio. Detalle: {e}", flush=True)

        print(f"\n[START ▶️] Forzando reinstalación del código del repositorio: {repo_url}", flush=True)
        cmd_force = [
            sys.executable, "-m", "pip", "install", "-q",
            "--upgrade", "--force-reinstall", "--no-deps", "--no-cache-dir",
            f"git+{repo_url}"
        ]
        try:
            subprocess.check_call(cmd_force)
            print(f"[SUCCESS ✅] Reinstalación del código completada.", flush=True)
        except subprocess.CalledProcessError as e:
            print(f"[ERROR ❌] Falló la reinstalación del código. Detalle: {e}", flush=True)

    def _detect_installed_package_name(before_set: set, repo_url: str) -> str:
        """
        Compara las distribuciones antes y después de la instalación:
        - before_set: conjunto de nombres de distribución (metadata["Name"]) previos.
        - repo_url: URL del repositorio (para intentar emparejar por substring).
        Devuelve el nombre de paquete (top-level) para importar.
        """
        # Obtenemos el conjunto actual de distribuciones instaladas
        after_set = {dist.metadata["Name"] for dist in metadata.distributions()}

        # Identificamos cuáles son las nuevas distribuciones
        nuevas = list(after_set - before_set)

        repo_name = repo_url.rstrip("/").split("/")[-1].lower()

        if not nuevas:
            # Si no hay distribuciones nuevas, intentamos emparejar alguna existente por substring
            candidatos = [dist.metadata["Name"] for dist in metadata.distributions() if repo_name in dist.metadata["Name"].lower()]
            if candidatos:
                elegida_dist = candidatos[0]
                print(f"[INFO ℹ️] No se detectó distribución nueva. Se asume '{elegida_dist}' para '{repo_name}'.", flush=True)
            else:
                print(f"[WARN ⚠️] No se detectó nueva distribución ni coincidencias para '{repo_name}'. Se usará '{repo_name}' por defecto.", flush=True)
                return repo_name
        else:
            # Si hay más de una nueva, intentamos filtrar por substring
            if len(nuevas) > 1:
                filtradas = [d for d in nuevas if repo_name in d.lower()]
                elegida_dist = filtradas[0] if filtradas else nuevas[0]
                print(f"[INFO ℹ️] Se detectaron múltiples distribuciones nuevas {nuevas}. Se elige '{elegida_dist}'.", flush=True)
            else:
                elegida_dist = nuevas[0]
            print(f"[INFO ℹ️] Se detectó la nueva distribución '{elegida_dist}'.", flush=True)

        # Ahora obtenemos el top-level package desde la metadata
        try:
            dist_info = metadata.distribution(elegida_dist)
            top_level_txt = dist_info.read_text("top_level.txt")
            if top_level_txt:
                top_levels = [line.strip() for line in top_level_txt.splitlines() if line.strip()]
                paquete = top_levels[0] if top_levels else elegida_dist
                print(f"[INFO ℹ️] Para la distribución '{elegida_dist}', se detectó el paquete top-level: '{paquete}'", flush=True)
                return paquete
            else:
                print(f"[WARN ⚠️] 'top_level.txt' vacío para '{elegida_dist}'. Se usará '{elegida_dist}' como paquete.", flush=True)
                return elegida_dist
        except Exception as e:
            # Si no existe top_level.txt o falla, devolvemos el nombre de la distribución
            print(f"[ERROR ❌] No se pudo leer 'top_level.txt' de '{elegida_dist}': {e}. Usando '{elegida_dist}'.", flush=True)
            return elegida_dist

    def _list_functions(package) -> None:
        """
        Recorre todos los módulos y submódulos del paquete e imprime las funciones definidas en cada uno.
        """
        print(f"\n[START ▶️] Funciones en el paquete: {package.__name__}", flush=True)
        for finder, module_name, is_pkg in pkgutil.walk_packages(package.__path__, package.__name__ + "."):
            try:
                modulo = importlib.import_module(module_name)
                funciones = [
                    nombre for nombre, objeto in inspect.getmembers(modulo, inspect.isfunction)
                    if inspect.getmodule(objeto) == modulo
                ]
                print(f"\n Módulo: {module_name}", flush=True)
                if funciones:
                    for funcion in funciones:
                        print("  -", funcion, flush=True)
                else:
                    print("  (No se encontraron funciones definidas)", flush=True)
            except Exception as e:
                print(f"[ERROR ❌] Falló al importar el módulo {module_name}: {e}", flush=True)

    # Mensaje de inicio del proceso global
    print("🔹🔹🔹 Instalación de Repositorios DPM y listado de funciones 🔹🔹🔹", flush=True)

    # Validación de la configuración
    github_repo_url_list = _validate_config(config)

    for repo_url in github_repo_url_list:
        # Tomamos snapshot de distribuciones antes de instalar
        antes = {dist.metadata["Name"] for dist in metadata.distributions()}

        # Instalamos el repositorio en dos pasos
        _install_repo(repo_url)

        # Detectamos automáticamente el nombre del paquete instalado
        package_name = _detect_installed_package_name(antes, repo_url)

        # Importamos el paquete y listamos sus funciones
        try:
            print(f"\n\n🔹🔹🔹 Importando el paquete: {package_name} 🔹🔹🔹", flush=True)
            if package_name in sys.modules:
                del sys.modules[package_name]
            importlib.invalidate_caches()
            paquete = importlib.import_module(package_name)
            _list_functions(paquete)
        except Exception as e:
            print(f"❌ [ERROR] Falló al importar el paquete '{package_name}': {e}", flush=True)

    print("\n\n🔹🔹🔹 [FINISHED ✅] Proceso completado. 🔹🔹🔹", flush=True)


Función ini_install_libraries cargada correctamente.
Función ini_load_dpm_libs cargada correctamente.
Función ini_environment_identification cargada correctamente.
Función ini_google_drive_instalation cargada correctamente.


In [ ]:
# @title IDENTIFICACION DE ENTORNO, INSTALACIÓN GOOGLE DRIVE

# Detectar el entorno de ejecución
ini_environment_identificated = ini_environment_identification()
print(f"[INFO ℹ️] Entorno detectado: {ini_environment_identificated}", flush=True)

GCP_json_keyfile_local = r"C:/api_keys/XXX.json"
GCP_json_keyfile_colab = "/content/drive/MyDrive/ANIMUM DIRECCION/DIRECCION BI/NOTEBOOKS/api_keys/animum-dev-apps-google-colab.json"
GCP_json_keyfile_GCP_secret_id = "notebook-vm"

# Montar Google Drive si entorno_identificado_str es Colab
params = {"entorno_identificado_str": ini_environment_identificated}
ini_google_drive_instalation(params)

[INFO ℹ️] Entorno detectado: COLAB
Mounted at /content/drive
[INFO ℹ️] Google Drive montado correctamente.


In [ ]:
# @title INSTALACION DE LIBRERIAS
from IPython import get_ipython
import os
packages = [
    {
        "name": "whisper",
        "import_name": "whisper",
        "install_cmd": "pip install git+https://github.com/openai/whisper.git" # Instalación desde GitHub:
    },
    # {
    #     "name": "ffmpeg-python",
    #     "import_name": "ffmpeg",
    #     "pip_name": "ffmpeg-python"
    # },
    # {
    #     "name": "FFmpeg",  # Paquete del sistema
    #     "is_system": True,
    #     "check_cmd": "ffmpeg -version",
    #     "install_cmds": [
    #         "apt-get update",
    #         "apt-get install -y ffmpeg",
    #         "ffmpeg -version"
    #     ]
    # },

    {"name": "rapidfuzz", "import_name": "rapidfuzz", "pip_name": "rapidfuzz"},
    {"name": "pycountry", "import_name": "pycountry", "pip_name": "pycountry"},
    {"name": "phonenumbers", "import_name": "phonenumbers", "pip_name": "phonenumbers"},
    {"name": "deep_translator", "import_name": "deep_translator", "pip_name": "deep_translator"},
    {"name": "requests", "import_name": "requests", "pip_name": "requests"},
    {"name": "beautifulsoup4", "import_name": "bs4", "pip_name": "beautifulsoup4"},
    {"name": "googletrans", "import_name": "googletrans", "pip_name": "googletrans", "version": "4.0.0-rc1"},
]

# Ejecuta la función para cada paquete
for pkg in packages:
    ini_install_libraries(pkg)



[START ▶️] Verificando instalación de whisper...
[INSTALLATION [INFO ℹ️]] whisper no está instalado. Procediendo con la instalación...
[INSTALLATION [COMMAND ▶️]] Ejecutando comando personalizado: pip install git+https://github.com/openai/whisper.git
[END [FINISHED ✅]] Proceso de instalación finalizado para whisper.


[START ▶️] Verificando instalación de rapidfuzz...
[INSTALLATION [INFO ℹ️]] rapidfuzz no está instalado. Procediendo con la instalación...
[INSTALLATION [COMMAND ▶️]] Ejecutando: pip install --upgrade rapidfuzz
[END [FINISHED ✅]] Proceso de instalación finalizado para rapidfuzz.


[START ▶️] Verificando instalación de pycountry...
[INSTALLATION [INFO ℹ️]] pycountry no está instalado. Procediendo con la instalación...
[INSTALLATION [COMMAND ▶️]] Ejecutando: pip install --upgrade pycountry
[END [FINISHED ✅]] Proceso de instalación finalizado para pycountry.


[START ▶️] Verificando instalación de phonenumbers...
[INSTALLATION [INFO ℹ️]] phonenumbers no está instalado. Proc

In [ ]:
# @title IMPORTACIÓN DE LIBRERÍAS DPM
import sys
import os
import importlib
import datetime
import inspect
import pandas as pd



# Configuración para importar librerías personalizadas
config = [
    {
        "module_host": "github",
        # Se utiliza la URL raw para obtener el contenido real del archivo
        "module_path": "https://raw.githubusercontent.com/DavidP0011/utils/main/dpm_tables.py",
        "selected_functions_list": []
    },
    {
        "module_host": "github",
        # Se utiliza la URL raw para obtener el contenido real del archivo
        "module_path": "https://raw.githubusercontent.com/DavidP0011/utils/main/dpm_GCP_utils.py",
        "selected_functions_list": []
    },
    {
        "module_host": "github",
        "module_path": "https://raw.githubusercontent.com/DavidP0011/utils/main/dpm_apps_utils.py",
        "selected_functions_list": []
    }
]

# Cargar las librerías personalizadas
ini_load_dpm_libs(config)



🔹🔹🔹 [START ▶️] Iniciando carga de módulo dpm_tables.py 🔹🔹🔹

[EXTRACTION [START ▶️]] Descargando módulo desde GitHub: https://raw.githubusercontent.com/DavidP0011/utils/main/dpm_tables.py
[EXTRACTION [SUCCESS ✅]] Archivo descargado y guardado en: /tmp/dpm_tables.py
[LOAD [START ▶️]] Importando módulo: dpm_tables
[LOAD [SUCCESS ✅]] Módulo 'dpm_tables' importado correctamente.

[METRICS [INFO 📊]] Informe de carga del módulo:
  - Módulo: dpm_tables
  - Ruta: /tmp/dpm_tables.py
  - Fecha de última modificación (último commit en GitHub o mod. local): 2025-03-11 17:42:43+01:00
  - Objetos importados:
      • fields_name_format (function): Formatea nombres de campos de datos según configuraciones específicas.
      • table_DF_to_various_targets (function): Escribe un DataFrame en distintos destinos (archivo local, Google Sheets, BigQuery o GCS)
      • table_various_sources_to_DF (function): Extrae datos desde distintos orígenes (archivo, Google Sheets, BigQuery o GCS) y los convierte en un D

## GBQ DATASETS SCHEMA

In [ ]:
# @title GBQ INFO GLOBAL


# Configuración
params_dic = {
    "spreadsheet_source_table_id": "1aJCGTJtDu_ODqBc4zUcrpQ-q6PE_HN0rO4mwYMIhCXw",
    "spreadsheet_source_table_worksheet_name": "DATA",

    "ini_environment_identificated": ini_environment_identificated,
    "json_keyfile_GCP_secret_id": GCP_json_keyfile_GCP_secret_id,
    "json_keyfile_colab": GCP_json_keyfile_colab
}



full_info_from_GBQ_df = table_various_sources_to_DF(params_dic)
display(full_info_from_GBQ_df)

[AUTHENTICATION [INFO] 🔐] Entorno local/Colab detectado. Usando json_keyfile_colab.
[EXTRACTION [START ⏳]] Extrayendo datos de Google Sheets...
[EXTRACTION [SUCCESS ✅]] Datos extraídos con éxito de la hoja 'DATA'.


,project_id,dataset_id,table_name,field_name,field_type,num_rows,num_columns,size_mb,fecha_actualizacion_GBQ,fecha_actualizacion_df
0,animum-dev-datawarehouse,IMDb_01raw_01,name_basics_raw,nconst,STRING,14235647,6,865.54,10/03/2025 19:43:37,11/03/2025 17:00:05
1,animum-dev-datawarehouse,IMDb_01raw_01,name_basics_raw,primaryName,STRING,14235647,6,865.54,10/03/2025 19:43:37,11/03/2025 17:00:05
2,animum-dev-datawarehouse,IMDb_01raw_01,name_basics_raw,birthYear,INTEGER,14235647,6,865.54,10/03/2025 19:43:37,11/03/2025 17:00:05
3,animum-dev-datawarehouse,IMDb_01raw_01,name_basics_raw,deathYear,STRING,14235647,6,865.54,10/03/2025 19:43:37,11/03/2025 17:00:05
4,animum-dev-datawarehouse,IMDb_01raw_01,name_basics_raw,primaryProfession,STRING,14235647,6,865.54,10/03/2025 19:43:37,11/03/2025 17:00:05
...,...,...,...,...,...,...,...,...,...,...
9688,animum-dev-datawarehouse,vl_01raw_01,MA_PROVINCIAS,Hora_creacion,DATETIME,1033,10,0.06,10/03/2025 5:22:56,11/03/2025 17:00:05
9689,animum-dev-datawarehouse,vl_01raw_01,MA_PROVINCIAS,Usuario_creacion,STRING,1033,10,0.06,10/03/2025 5:22:56,11/03/2025 17:00:05
9690,animum-dev-datawarehouse,vl_01raw_01,MA_PROVINCIAS,Fecha_modificacion,DATETIME,1033,10,0.06,10/03/2025 5:22:56,11/03/2025 17:00:05
9691,animum-dev-datawarehouse,vl_01raw_01,MA_PROVINCIAS,Hora_modificacion,DATETIME,1033,10,0.06,10/03/2025 5:22:56,11/03/2025 17:00:05


In [ ]:
# @title GBQ DATASETS
print("Datasets disponibles:\n")
for dataset in full_info_from_GBQ_df['dataset_id'].unique():
    print(f"{dataset}")

Datasets disponibles:

IMDb_01raw_01
IMDb_02st_01
IMDb_raw_01
IMDb_staging_01
cd2_01raw_01
facebook_ads_raw_v01
facebook_ads_raw_v01_facebook_ads
facebook_ads_raw_v01_facebook_ads_source
fivetran_metadata
fivetran_metadata_fivetran_platform
fivetran_metadata_stg_fivetran_platform
google_ads_raw_01
google_ads_raw_01_google_ads
google_ads_raw_01_google_ads_source
hubspot_raw_v01
hubspot_raw_v01_hubspot
hubspot_raw_v01_stg_hubspot
mkt_02st_01
mkt_03BI_01
tablas_mapeo
tp_02st_01
tp_03bi_01
vl_01raw_01


# EJECUCIONES

In [ ]:
# @title OBTENCION DF CON RUTAS


# Configuración
config = {
    "spreadsheet_source_table_id": "1_WDFCgVu4dtFDgCcW7Yzhx6-gUlQeJiBmIrwzA5agGI",
    "spreadsheet_source_table_worksheet_name": "SOURCE",

    "ini_environment_identificated": ini_environment_identificated,
    "json_keyfile_local": GCP_json_keyfile_local,
    "json_keyfile_colab": GCP_json_keyfile_colab,
    "json_keyfile_GCP_secret_id": GCP_json_keyfile_GCP_secret_id,
}

import os
import pandas as pd

df = table_various_sources_to_DF(config)

display(df)

[AUTHENTICATION [INFO] 🔐] Entorno local/Colab detectado. Usando json_keyfile_colab.
[EXTRACTION [START ⏳]] Extrayendo datos de Google Sheets...
[EXTRACTION [SUCCESS ✅]] Datos extraídos con éxito de la hoja 'SOURCE'.


,file_path,Terminado
0,/content/drive/Shareddrives/AREA ACADEMICO/MOC...,#N/A
1,/content/drive/Shareddrives/AREA ACADEMICO/MOC...,#N/A
2,/content/drive/Shareddrives/AREA ACADEMICO/MOC...,#N/A
3,/content/drive/Shareddrives/AREA ACADEMICO/MOC...,#N/A
4,/content/drive/Shareddrives/AREA ACADEMICO/MOC...,#N/A
...,...,...
415,/content/drive/Shareddrives/AREA ACADEMICO/MOC...,#N/A
416,/content/drive/Shareddrives/AREA ACADEMICO/MOC...,#N/A
417,/content/drive/Shareddrives/AREA ACADEMICO/MOC...,#N/A
418,/content/drive/Shareddrives/AREA ACADEMICO/MOC...,#N/A


In [ ]:
# @title TRANSCRIPCIÓN Y SUBIDA A SPREADSHEET

config = {
    "source_files_path_table_df": df,
    "target_files_path_table_spreadsheet_url": "https://docs.google.com/spreadsheets/d/1_WDFCgVu4dtFDgCcW7Yzhx6-gUlQeJiBmIrwzA5agGI",
    "target_files_path_table_spreadsheet_worksheet": "ResponseAPI",
    "field_name_for_file_path": "file_path",

    "whisper_model_size": "large-v2",
    "whisper_language": "es",  # Ejemplo: español

    "ini_environment_identificated": ini_environment_identificated,
    "json_keyfile_local": GCP_json_keyfile_local,
    "json_keyfile_colab": GCP_json_keyfile_colab,
    "json_keyfile_GCP_secret_id": GCP_json_keyfile_GCP_secret_id,
}

# Llamar a la función con manejo de errores (la función ya incorpora try/except internamente)
df_to_whisper_transcribe_to_spreadsheet(config)



🔹🔹🔹 [START ▶️] Autenticando con Google Sheets 🔹🔹🔹
[AUTH SUCCESS ✅] Autenticación exitosa.
🔹🔹🔹 [START ▶️] Preparando hoja destino: ResponseAPI 🔹🔹🔹
[SHEET SUCCESS ✅] Hoja destino preparada y encabezados definidos.
🔹🔹🔹 [START ▶️] Cargando modelo Whisper 'large-v2' 🔹🔹🔹


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


[MODEL SUCCESS ✅] Modelo Whisper cargado.
[DATA INFO ℹ️] DataFrame fuente cargado. Total filas: 420
[PROCESSING 🔄] (1/420) Transcribiendo: /content/drive/Shareddrives/AREA ACADEMICO/MOCADI/Sesiones en Directo/Bienvenida Animum.mp4 (idioma='es')
[WRITE SUCCESS ✅] Fila 1 escrita correctamente.
[PROCESSING 🔄] (2/420) Transcribiendo: /content/drive/Shareddrives/AREA ACADEMICO/MOCADI/Sesiones en Directo/T3 2024 Bienvenida TFP TPP .mp4 (idioma='es')
[WRITE SUCCESS ✅] Fila 2 escrita correctamente.
[PROCESSING 🔄] (3/420) Transcribiendo: /content/drive/Shareddrives/AREA ACADEMICO/MOCADI/Sesiones en Directo/T3 2024 Sesión sobre la Industria - TFP.mp4 (idioma='es')
[WRITE SUCCESS ✅] Fila 3 escrita correctamente.
[PROCESSING 🔄] (4/420) Transcribiendo: /content/drive/Shareddrives/AREA ACADEMICO/MOCADI/Sesiones en Directo/T4 2024 Sesión sobre la Industria - TFP.mp4 (idioma='es')
[WRITE SUCCESS ✅] Fila 4 escrita correctamente.
[PROCESSING 🔄] (5/420) Transcribiendo: /content/drive/Shareddrives/AREA 